<a href="https://colab.research.google.com/github/aymensrihi/deep-learning-projects/blob/main/WAiT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import numpy as np
from functools import partial
import random
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import math

In [12]:
from google.colab import files

uploaded = files.upload()


Saving DR.tar.gz to DR.tar.gz


In [13]:
!ls -la

total 669188
drwxr-xr-x 1 root root      4096 Dec 24 12:51 .
drwxr-xr-x 1 root root      4096 Dec 24 11:25 ..
drwxr-xr-x 4 root root      4096 Dec 11 14:34 .config
-rw-r--r-- 1 root root 685226467 Dec 24 12:51 DR.tar.gz
drwxr-xr-x 1 root root      4096 Dec 11 14:34 sample_data


In [15]:
import os
!tar -xvzf DR.tar.gz


extracted_path = "/content/DR"
print(os.listdir(extracted_path))

Streaming output truncated to the last 5000 lines.
DR/messidor_2/0/20060412_58802_0200_PP.png
DR/messidor_2/0/IM000191.JPG
DR/messidor_2/0/20051020_43808_0100_PP.png
DR/messidor_2/0/20051214_51211_0100_PP.png
DR/messidor_2/0/IM002251.JPG
DR/messidor_2/0/20060410_39859_0200_PP.png
DR/messidor_2/0/20060411_62058_0200_PP.png
DR/messidor_2/0/IM002019.JPG
DR/messidor_2/0/20051021_57975_0100_PP.png
DR/messidor_2/0/20060523_45300_0100_PP.png
DR/messidor_2/0/IM003474.JPG
DR/messidor_2/0/20060410_43847_0200_PP.png
DR/messidor_2/0/IM003925.JPG
DR/messidor_2/0/20051130_54257_0400_PP.png
DR/messidor_2/0/IM002704.JPG
DR/messidor_2/0/20051214_42323_0100_PP.png
DR/messidor_2/0/20060523_49663_0100_PP.png
DR/messidor_2/0/IM002215.JPG
DR/messidor_2/0/IM004191.JPG
DR/messidor_2/0/20060523_49681_0100_PP.png
DR/messidor_2/0/IM003175.JPG
DR/messidor_2/0/IM003338.JPG
DR/messidor_2/0/IM000305.JPG
DR/messidor_2/0/20060523_49100_0100_PP.png
DR/messidor_2/0/20060410_44345_0200_PP.png
DR/messidor_2/0/20051205_579

In [16]:
extracted_path = "/content/DR"
datasets = os.listdir(extracted_path)
image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif')

for dataset in datasets:
    dataset_path = os.path.join(extracted_path, dataset)
    total_images = 0
    for root, dirs, files in os.walk(dataset_path):
        for f in files:
            if f.lower().endswith(image_extensions):
                total_images += 1
    print(f"{dataset}: {total_images} images")

eyepacs: 88702 images
messidor_2: 1744 images
messidor: 1200 images
aptos: 3662 images


In [17]:
print("number of images in eyepacs")
!ls -l /content/DR/eyepacs/0 | grep -v '^d' | wc -l
!ls -l /content/DR/eyepacs/1 | grep -v '^d' | wc -l

print("number of images in messidor2")
!ls -l /content/DR/messidor_2/0 | grep -v '^d' | wc -l
!ls -l /content/DR/messidor_2/1 | grep -v '^d' | wc -l
!ls -l /content/DR/messidor_2/2 | grep -v '^d' | wc -l
!ls -l /content/DR/messidor_2/3 | grep -v '^d' | wc -l
!ls -l /content/DR/messidor_2/4 | grep -v '^d' | wc -l
print("number of images in messidor")
!ls -l /content/DR/messidor/0 | grep -v '^d' | wc -l
!ls -l /content/DR/messidor/1 | grep -v '^d' | wc -l
!ls -l /content/DR/messidor/2 | grep -v '^d' | wc -l
!ls -l /content/DR/messidor/3 | grep -v '^d' | wc -l
!ls -l /content/DR/messidor/4 | grep -v '^d' | wc -l
print("number of images in aptos")
!ls -l /content/DR/aptos/0 | grep -v '^d' | wc -l
!ls -l /content/DR/aptos/1 | grep -v '^d' | wc -l
!ls -l /content/DR/aptos/2 | grep -v '^d' | wc -l
!ls -l /content/DR/aptos/3 | grep -v '^d' | wc -l
!ls -l /content/DR/aptos/4 | grep -v '^d' | wc -l

number of images in eyepacs
65344
6206
number of images in messidor2
1018
271
348
76
36
number of images in messidor
547
154
248
255
1
number of images in aptos
1806
371
1000
194
296


In [2]:
def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    """Truncated normal initialization"""
    def norm_cdf(x):
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    with torch.no_grad():
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)
        tensor.uniform_(2 * l - 1, 2 * u - 1)
        tensor.erfinv_()
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)
        tensor.clamp_(min=a, max=b)
        return tensor

def to_2tuple(x):
    return (x, x) if isinstance(x, int) else tuple(x)

class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample"""
    def __init__(self, drop_prob=None):
        super().__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        output = x.div(keep_prob) * random_tensor
        return output

class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x, return_attention=False):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attncpy = attn
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)

        if return_attention:
            return x, attncpy
        return x

class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None,
                 drop=0., attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
                             attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x, return_attention=False):
        if return_attention:
            attn, attncpy = self.attn(self.norm1(x), return_attention=True)
        else:
            attn = self.attn(self.norm1(x))

        x = x + self.drop_path(attn)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        if return_attention:
            return x, attncpy
        return x

class PatchEmbed(nn.Module):
    """Image to Patch Embedding"""
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

class VisionTransformer(nn.Module):
    """Vision Transformer with SPSD support - returns layer-wise outputs"""
    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=1000,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4., qkv_bias=False,
                 qk_scale=None, drop_rate=0., attn_drop_rate=0., drop_path_rate=0.,
                 norm_layer=nn.LayerNorm):
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim

        self.patch_embed = PatchEmbed(img_size=img_size, patch_size=patch_size,
                                     in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]
        self.blocks = nn.ModuleList([
            Block(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias,
                 qk_scale=qk_scale, drop=drop_rate, attn_drop=attn_drop_rate,
                 drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        self.head = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()

        trunc_normal_(self.pos_embed, std=.02)
        trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward_features(self, x):
        """
        CRITICAL: Returns layer-wise CLS token features (NOT predictions)
        This is required for SPSD to work correctly
        """
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        # Collect CLS token from each layer
        layer_wise_tokens = []
        for blk in self.blocks:
            x = blk(x)
            layer_wise_tokens.append(x)

        # Apply normalization to each layer's output
        layer_wise_tokens = [self.norm(x) for x in layer_wise_tokens]
        # Return only CLS tokens (first token from each layer)
        return [x[:, 0] for x in layer_wise_tokens]

    def forward(self, x, return_feat=False):
        """
        Returns list of predictions from all layers
        This is the key difference from standard ViT
        """
        list_out = self.forward_features(x)
        features = list_out
        # Apply classifier head to each layer's CLS token
        x = [self.head(feat) for feat in list_out]

        if return_feat:
            return x, features
        return x

In [3]:
class MultipleEnvironmentImageFolder(Dataset):
    """Dataset that loads images from multiple domains"""
    def __init__(self, root, test_envs, augment, hparams):
        super().__init__()
        environments = [f.name for f in os.scandir(root) if f.is_dir()]
        environments = sorted(environments)

        transform_list = [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]

        if augment:
            transform_list.insert(1, transforms.RandomHorizontalFlip())
            transform_list.insert(1, transforms.RandomRotation(15))
            transform_list.insert(1, transforms.ColorJitter(brightness=0.4, contrast=0.4))

        self.transform = transforms.Compose(transform_list)
        self.datasets = []

        for i, env_name in enumerate(environments):
            path = os.path.join(root, env_name)
            env_dataset = ImageFolderDataset(path, self.transform)
            self.datasets.append(env_dataset)

        self.input_shape = (3, 224, 224)
        self.num_classes = len(self.datasets[0].classes) if self.datasets else 5

In [4]:
class ImageFolderDataset(Dataset):
    """Simple image folder dataset"""
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.classes = sorted([d.name for d in os.scandir(root) if d.is_dir()])
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(root, class_name)
            class_idx = self.class_to_idx[class_name]
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.samples.append((os.path.join(class_dir, img_name), class_idx))
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, target = self.samples[idx]
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, target

In [5]:
class DR(MultipleEnvironmentImageFolder):
    CHECKPOINT_FREQ = 100
    ENVIRONMENTS = ["aptos", "eyepacs", "messidor", "messidor_2"]
    def __init__(self, root, test_envs, hparams):
        self.dir = os.path.join(root, "DR/")
        super().__init__(self.dir, test_envs, hparams['data_augmentation'], hparams)

In [6]:
class ERM_SPSDViT(nn.Module):
    """
    SPSD Vision Transformer Algorithm
    Paper: "Generalizing to Unseen Domains in Diabetic Retinopathy Classification"

    Key components from paper:
    1. Random Block Selection (RBS): Randomly select intermediate classifier
    2. Progressive Soft Pseudo Labeling (PSPL): Adaptive mixing of one-hot and predictions
    3. KL Divergence Loss: Between softened predictions from final and intermediate blocks
    """
    def __init__(self, input_shape, num_classes, num_domains, hparams):
        super().__init__()
        self.hparams = hparams
        self.alpha_rb_loss = hparams['RB_loss_weight']  # λ in paper (default: 0.7)
        self.alpha_KL_temp = hparams['KL_Div_Temperature']  # Not used in SPSD, kept for compatibility
        self.beta_T = hparams['alpha_T']  # β_T in paper (default: 0.8)
        self.n_steps = hparams['n_steps']  # Total training steps
        self.step_count = 0
        self.n_classes = num_classes

        print(f"SPSD-ViT Configuration:")
        print(f"  λ (RB_loss_weight): {self.alpha_rb_loss}")
        print(f"  β_T (alpha_T): {self.beta_T}")
        print(f"  Total steps: {self.n_steps}")

        # Create model (DeiT-Small-like architecture)
        self.network = VisionTransformer(
            img_size=224, patch_size=16, embed_dim=384, depth=12,
            num_heads=6, mlp_ratio=4, qkv_bias=True,
            num_classes=num_classes
        )

        self.optimizer = torch.optim.AdamW(
            self.network.parameters(),
            lr=hparams["lr"],
            weight_decay=hparams['weight_decay']
        )

    def update(self, minibatches):
        """
        Training update step with SPSD
        Paper Equation 3: L_KL with adaptive convex combination
        """
        all_x = torch.cat([x for x, y in minibatches])
        all_y = torch.cat([y for x, y in minibatches])

        # Compute β_t: Progressive coefficient (Equation in Section 3.2)
        # β_t = β_T × (t/T) where t is current step, T is total steps
        beta_t = self.beta_T * ((self.step_count + 1) / self.n_steps)
        beta_t = max(0.0, min(beta_t, self.beta_T))  # Clamp to [0, β_T]

        self.step_count += 1

        # Forward pass: Get predictions from all layers
        outputs = self.network(all_x)  # List of logits from each layer

        # Final classifier output (last layer)
        z = outputs[-1]

        # Random Block Selection (RBS): Select one intermediate block
        block_idx = random.randint(0, len(outputs) - 1)
        z_j = outputs[block_idx]

        # Create one-hot labels
        y_one_hot = torch.zeros(all_y.size(0), self.n_classes, device=all_x.device)
        y_one_hot.scatter_(1, all_y.unsqueeze(1), 1)

        # Adaptive Convex Combination (Section 3.2)
        # Soft prediction: β_t * z + (1 - β_t) * y
        soft_z = beta_t * z + (1 - beta_t) * y_one_hot
        soft_z_j = beta_t * z_j + (1 - beta_t) * y_one_hot

        # Apply softmax to get probability distributions
        soft_output = F.softmax(soft_z, dim=1)
        soft_output_rb = F.softmax(soft_z_j, dim=1)

        # Standard cross-entropy loss on final classifier
        base_loss = F.cross_entropy(z, all_y)

        # KL Divergence Loss (Paper Equation 3)
        # L_KL between softened predictions
        rb_loss = F.kl_div(
            torch.log(soft_output_rb + 1e-10),  # Add epsilon for numerical stability
            soft_output.detach(),  # Detach to avoid gradient through full network twice
            reduction='batchmean',
            log_target=False
        )

        # Total loss (Paper Equation 2): L = L_CE + λ * L_KL
        loss = base_loss + self.alpha_rb_loss * rb_loss

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return {
            'loss': loss.item(),
            'rb_loss': rb_loss.item(),
            'base_loss': base_loss.item(),
            'beta_t': beta_t
        }

    def predict(self, x):
        """Inference: Use only final classifier"""
        outputs = self.network(x)
        return outputs[-1]  # Return final layer output only

In [7]:
def default_hparams(algorithm='ERM_SPSDViT', dataset='DR'):
    """
    Default hyperparameters matching the paper
    From Table 10 and experimental setup
    """
    hparams = {
        'data_augmentation': True,
        'backbone': 'DeitSmall',
        'nonlinear_classifier': False,

        # SPSD-specific hyperparameters
        'RB_loss_weight': 0.7,  # λ - balances KL divergence loss
        'KL_Div_Temperature': 4.0,  # Legacy parameter (not used in SPSD)
        'alpha_T': 0.8,  # β_T - maximum mixing coefficient
        'n_steps': 5000,  # Total training steps

        # Optimization
        'lr': 5e-5,  # Learning rate (from paper)
        'weight_decay': 0.0,  # Weight decay
        'batch_size': 32  # Batch size (from paper)
    }
    return hparams

In [8]:
def accuracy(predictions, targets):
    """Calculate top-1 accuracy"""
    _, predicted = torch.max(predictions, 1)
    correct = (predicted == targets).sum().item()
    return correct / targets.size(0)

def train_epoch(algorithm, data_loader, device):
    """Train for one epoch"""
    algorithm.train()
    total_loss = 0
    total_base_loss = 0
    total_rb_loss = 0
    total_acc = 0
    n_batches = 0

    for batch_idx, minibatches in enumerate(data_loader):
        minibatches = [(x.to(device), y.to(device)) for x, y in minibatches]

        step_vals = algorithm.update(minibatches)

        # Calculate accuracy on final classifier
        all_x = torch.cat([x for x, y in minibatches])
        all_y = torch.cat([y for x, y in minibatches])
        with torch.no_grad():
            predictions = algorithm.predict(all_x)
            acc = accuracy(predictions, all_y)

        total_loss += step_vals['loss']
        total_base_loss += step_vals['base_loss']
        total_rb_loss += step_vals['rb_loss']
        total_acc += acc
        n_batches += 1

        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}: Loss={step_vals['loss']:.4f}, "
                  f"Base={step_vals['base_loss']:.4f}, RB={step_vals['rb_loss']:.4f}, "
                  f"Acc={acc:.4f}, β_t={step_vals['beta_t']:.4f}")

    return {
        'loss': total_loss / n_batches,
        'base_loss': total_base_loss / n_batches,
        'rb_loss': total_rb_loss / n_batches,
        'accuracy': total_acc / n_batches
    }

@torch.no_grad()
def evaluate(algorithm, data_loader, device):
    """Evaluate on validation/test set"""
    algorithm.eval()
    total_acc = 0
    total_samples = 0

    for batch in data_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)

        predictions = algorithm.predict(x)
        _, predicted = torch.max(predictions, 1)
        correct = (predicted == y).sum().item()

        total_acc += correct
        total_samples += y.size(0)

    return total_acc / total_samples

In [11]:
def main():
    """Example usage matching the paper's experimental setup"""
    print("=" * 80)
    print("SPSD Vision Transformer for Diabetic Retinopathy")
    print("Paper: Generalizing to Unseen Domains in Diabetic Retinopathy Classification")
    print("=" * 80)

    # Setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\nUsing device: {device}")

    # Hyperparameters from paper
    hparams = default_hparams()
    print("\nHyperparameters (from paper):")
    for k, v in hparams.items():
        print(f"  {k}: {v}")

    # Create algorithm
    num_classes = 5  # DR has 5 grades (0-4)
    num_domains = 4  # aptos, eyepacs, messidor, messidor_2
    input_shape = (3, 224, 224)

    algorithm = ERM_SPSDViT(input_shape, num_classes, num_domains, hparams)
    algorithm = algorithm.to(device)

    total_params = sum(p.numel() for p in algorithm.parameters())
    trainable_params = sum(p.numel() for p in algorithm.parameters() if p.requires_grad)

    print(f"\nModel Statistics:")
    print(f"  Total parameters: {total_params:,}")
    print(f"  Trainable parameters: {trainable_params:,}")

    print("\n" + "=" * 80)
    print("USAGE INSTRUCTIONS")
    print("=" * 80)
    print("")

In [ ]:
"""
SPSD Vision Transformer for Domain Generalization on Diabetic Retinopathy
Complete implementation with dataset loading and training pipeline
Paper: "Generalizing to Unseen Domains in Diabetic Retinopathy Classification"
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import random
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import math
from tqdm import tqdm

# ============================================================================
# DATASET SETUP (Dataset already extracted)
# ============================================================================

# ============================================================================
# VISION TRANSFORMER COMPONENTS
# ============================================================================

def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    """Truncated normal initialization"""
    def norm_cdf(x):
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    with torch.no_grad():
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)
        tensor.uniform_(2 * l - 1, 2 * u - 1)
        tensor.erfinv_()
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)
        tensor.clamp_(min=a, max=b)
        return tensor

def to_2tuple(x):
    return (x, x) if isinstance(x, int) else tuple(x)

class DropPath(nn.Module):
    def __init__(self, drop_prob=None):
        super().__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        return x.div(keep_prob) * random_tensor

class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None,
                 drop=0., attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
                             attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=1000,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4., qkv_bias=False,
                 qk_scale=None, drop_rate=0., attn_drop_rate=0., drop_path_rate=0.,
                 norm_layer=nn.LayerNorm):
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim

        self.patch_embed = PatchEmbed(img_size=img_size, patch_size=patch_size,
                                     in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]
        self.blocks = nn.ModuleList([
            Block(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias,
                 qk_scale=qk_scale, drop=drop_rate, attn_drop=attn_drop_rate,
                 drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        self.head = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()

        trunc_normal_(self.pos_embed, std=.02)
        trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward_features(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        layer_wise_tokens = []
        for blk in self.blocks:
            x = blk(x)
            layer_wise_tokens.append(x)

        layer_wise_tokens = [self.norm(x) for x in layer_wise_tokens]
        return [x[:, 0] for x in layer_wise_tokens]

    def forward(self, x):
        list_out = self.forward_features(x)
        x = [self.head(feat) for feat in list_out]
        return x

# ============================================================================
# DATASET CLASS
# ============================================================================

class DRDataset(Dataset):
    """Diabetic Retinopathy Dataset"""
    def __init__(self, root, domain_name, transform=None):
        self.root = os.path.join(root, domain_name)
        self.transform = transform
        self.classes = ['0', '1', '2', '3', '4']  # DR grades
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        self.samples = []
        image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif')

        for class_name in self.classes:
            class_dir = os.path.join(self.root, class_name)
            if not os.path.exists(class_dir):
                continue

            class_idx = self.class_to_idx[class_name]
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(image_extensions):
                    self.samples.append((os.path.join(class_dir, img_name), class_idx))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, target = self.samples[idx]
        try:
            img = Image.open(path).convert('RGB')
            if self.transform:
                img = self.transform(img)
            return img, target
        except Exception as e:
            print(f"Error loading {path}: {e}")
            # Return a random other sample
            return self.__getitem__(random.randint(0, len(self) - 1))

# ============================================================================
# SPSD ALGORITHM
# ============================================================================

class SPSDViT(nn.Module):
    """SPSD Vision Transformer with Progressive Soft Pseudo-Labeling"""
    def __init__(self, num_classes=5, hparams=None):
        super().__init__()
        if hparams is None:
            hparams = default_hparams()

        self.hparams = hparams
        self.lambda_ = hparams['RB_loss_weight']
        self.beta_T = hparams['alpha_T']
        self.n_steps = hparams['n_steps']
        self.step_count = 0
        self.n_classes = num_classes

        # DeiT-Small architecture
        self.network = VisionTransformer(
            img_size=224, patch_size=16, embed_dim=384, depth=12,
            num_heads=6, mlp_ratio=4, qkv_bias=True,
            num_classes=num_classes
        )

        self.optimizer = torch.optim.AdamW(
            self.network.parameters(),
            lr=hparams["lr"],
            weight_decay=hparams['weight_decay']
        )

    def update(self, x, y):
        """Single training step with SPSD"""
        # Compute progressive beta_t
        beta_t = self.beta_T * ((self.step_count + 1) / self.n_steps)
        beta_t = max(0.0, min(beta_t, self.beta_T))
        self.step_count += 1

        # Forward pass
        outputs = self.network(x)
        z = outputs[-1]  # Final classifier

        # Random block selection
        block_idx = random.randint(0, len(outputs) - 1)
        z_j = outputs[block_idx]

        # One-hot labels
        y_one_hot = torch.zeros(y.size(0), self.n_classes, device=x.device)
        y_one_hot.scatter_(1, y.unsqueeze(1), 1)

        # Adaptive convex combination (PSPL)
        soft_z = beta_t * z + (1 - beta_t) * y_one_hot
        soft_z_j = beta_t * z_j + (1 - beta_t) * y_one_hot

        soft_output = F.softmax(soft_z, dim=1)
        soft_output_rb = F.softmax(soft_z_j, dim=1)

        # Losses
        base_loss = F.cross_entropy(z, y)
        rb_loss = F.kl_div(
            torch.log(soft_output_rb + 1e-10),
            soft_output.detach(),
            reduction='batchmean',
            log_target=False
        )

        loss = base_loss + self.lambda_ * rb_loss

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return {
            'loss': loss.item(),
            'base_loss': base_loss.item(),
            'rb_loss': rb_loss.item(),
            'beta_t': beta_t
        }

    def predict(self, x):
        outputs = self.network(x)
        return outputs[-1]

# ============================================================================
# TRAINING UTILITIES
# ============================================================================

def default_hparams():
    return {
        'data_augmentation': True,
        'RB_loss_weight': 0.7,
        'KL_Div_Temperature': 4.0,
        'alpha_T': 0.8,
        'n_steps': 5000,
        'lr': 5e-5,
        'weight_decay': 0.0,
        'batch_size': 32
    }

def get_transforms(augment=True):
    """Get data transforms"""
    transform_list = [transforms.Resize((224, 224))]

    if augment:
        transform_list.extend([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.4, contrast=0.4)
        ])

    transform_list.extend([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    return transforms.Compose(transform_list)

@torch.no_grad()
def evaluate(model, data_loader, device):
    """Evaluate model"""
    model.eval()
    correct = 0
    total = 0

    for x, y in data_loader:
        x, y = x.to(device), y.to(device)
        predictions = model.predict(x)
        _, predicted = torch.max(predictions, 1)
        correct += (predicted == y).sum().item()
        total += y.size(0)

    return correct / total if total > 0 else 0.0

def train_multi_source_dg(data_root, test_domain, num_epochs=10, hparams=None):
    """
    Multi-Source Domain Generalization Training

    Args:
        data_root: Path to DR dataset root
        test_domain: Domain to test on (e.g., 'messidor_2')
        num_epochs: Number of training epochs
        hparams: Hyperparameters dictionary
    """
    if hparams is None:
        hparams = default_hparams()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}\n")

    # Setup domains
    all_domains = ['aptos', 'eyepacs', 'messidor', 'messidor_2']
    train_domains = [d for d in all_domains if d != test_domain]

    print(f"Training domains: {train_domains}")
    print(f"Test domain: {test_domain}\n")

    # Create datasets
    train_transform = get_transforms(augment=True)
    test_transform = get_transforms(augment=False)

    train_datasets = [DRDataset(data_root, d, train_transform) for d in train_domains]
    test_dataset = DRDataset(data_root, test_domain, test_transform)

    # Create dataloaders
    train_loaders = [
        DataLoader(ds, batch_size=hparams['batch_size'], shuffle=True, num_workers=2)
        for ds in train_datasets
    ]
    test_loader = DataLoader(test_dataset, batch_size=hparams['batch_size'],
                            shuffle=False, num_workers=2)

    print(f"Training samples: {sum(len(ds) for ds in train_datasets)}")
    print(f"Test samples: {len(test_dataset)}\n")

    # Create model
    model = SPSDViT(num_classes=5, hparams=hparams).to(device)

    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}\n")

    # Training loop
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        epoch_base_loss = 0
        epoch_rb_loss = 0
        n_batches = 0

        # Create iterators for all training domains
        train_iters = [iter(loader) for loader in train_loaders]
        max_batches = max(len(loader) for loader in train_loaders)

        pbar = tqdm(range(max_batches), desc=f"Epoch {epoch+1}/{num_epochs}")

        for batch_idx in pbar:
            # Sample from each domain
            all_x, all_y = [], []
            for train_iter, loader in zip(train_iters, train_loaders):
                try:
                    x, y = next(train_iter)
                except StopIteration:
                    train_iter = iter(loader)
                    x, y = next(train_iter)
                all_x.append(x)
                all_y.append(y)

            x = torch.cat(all_x).to(device)
            y = torch.cat(all_y).to(device)

            # Training step
            step_vals = model.update(x, y)

            epoch_loss += step_vals['loss']
            epoch_base_loss += step_vals['base_loss']
            epoch_rb_loss += step_vals['rb_loss']
            n_batches += 1

            pbar.set_postfix({
                'loss': f"{step_vals['loss']:.4f}",
                'β_t': f"{step_vals['beta_t']:.4f}"
            })

        # Evaluation
        test_acc = evaluate(model, test_loader, device)

        print(f"\nEpoch {epoch+1}/{num_epochs}:")
        print(f"  Loss: {epoch_loss/n_batches:.4f}")
        print(f"  Base Loss: {epoch_base_loss/n_batches:.4f}")
        print(f"  RB Loss: {epoch_rb_loss/n_batches:.4f}")
        print(f"  Test Accuracy ({test_domain}): {test_acc*100:.2f}%")

        if test_acc > best_acc:
            best_acc = test_acc
            print(f"  ✓ New best accuracy!")
        print()

    print(f"Training completed! Best test accuracy: {best_acc*100:.2f}%")
    return model, best_acc

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("=" * 80)
    print("SPSD-ViT: Diabetic Retinopathy Domain Generalization")
    print("=" * 80)

    # Dataset path (already extracted)
    data_path = "/content/DR"  # Change this to your dataset path

    # Verify dataset exists
    if not os.path.exists(data_path):
        print(f"ERROR: Dataset not found at {data_path}")
        print("Please update the data_path variable to point to your DR folder")
        exit(1)

    print(f"\n✓ Dataset found at: {data_path}")
    print(f"✓ Domains available: {sorted(os.listdir(data_path))}\n")

    # Training configuration
    test_domain = 'messidor_2'  # Test on Messidor 2
    num_epochs = 10  # Adjust as needed

    hparams = default_hparams()
    hparams['n_steps'] = 5000  # Adjust based on your dataset size

    print(f"Configuration:")
    print(f"  Test domain: {test_domain}")
    print(f"  Training epochs: {num_epochs}")
    print(f"  Batch size: {hparams['batch_size']}")
    print(f"  Learning rate: {hparams['lr']}")
    print(f"  λ (RB loss weight): {hparams['RB_loss_weight']}")
    print(f"  β_T (max mixing): {hparams['alpha_T']}\n")

    # Train model
    model, best_acc = train_multi_source_dg(
        data_root=data_path,
        test_domain=test_domain,
        num_epochs=num_epochs,
        hparams=hparams
    )

    print("\n" + "=" * 80)
    print(f"✓ Training complete!")
    print(f"✓ Best accuracy on {test_domain}: {best_acc*100:.2f}%")
    print("=" * 80)

    # Optional: Test on all other domains
    print("\n" + "=" * 80)
    print("Testing on all domains:")
    print("=" * 80)

    all_domains = ['aptos', 'eyepacs', 'messidor', 'messidor_2']
    test_transform = get_transforms(augment=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    results = {}
    for domain in all_domains:
        test_dataset = DRDataset(data_path, domain, test_transform)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
        acc = evaluate(model, test_loader, device)
        results[domain] = acc
        print(f"  {domain}: {acc*100:.2f}%")

    avg_acc = sum(results.values()) / len(results)
    print(f"\n  Average accuracy: {avg_acc*100:.2f}%")
    print("=" * 80)

SPSD-ViT: Diabetic Retinopathy Domain Generalization

✓ Dataset found at: /content/DR
✓ Domains available: ['aptos', 'eyepacs', 'messidor', 'messidor_2']

Configuration:
  Test domain: messidor_2
  Training epochs: 10
  Batch size: 32
  Learning rate: 5e-05
  λ (RB loss weight): 0.7
  β_T (max mixing): 0.8

Using device: cpu

Training domains: ['aptos', 'eyepacs', 'messidor']
Test domain: messidor_2

Training samples: 93564
Test samples: 1744

Model parameters: 21,667,589



Epoch 1/10:   1%|          | 34/2772 [28:13<36:52:06, 48.48s/it, loss=1.3084, β_t=0.0054]